# This file is used to scrape for wrestler data.

In [2]:
import sys
!{sys.executable} -m pip install lxml 
!{sys.executable} -m pip install html5lib  

In [3]:
import matplotlib as mpltlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime


import requests, re, json
from bs4 import BeautifulSoup
import math

import rank_helper
import kimarite_helper

In [4]:
def get_df_by_id(id):
    
    URL = f"http://sumodb.sumogames.de/Rikishi.aspx?r={id}"
    basepage = requests.get(URL)
    soup = BeautifulSoup(basepage.content, 'html.parser')
    
    try:
        highest = soup.find('table', class_='rikishidata')
        highest = highest.find('table', class_='rikishidata')#.find('tr').findAll('td')[1].contents[0]
        highest = highest.find('tr').findAll('td')[1].contents[0]
        highest = str(highest.split()[0]).replace('-', '')
    except:
        print(f"{id} skipped prelim")
        return
    
    
    if not rank_helper.rank_is_top(highest, True):
        print(f"{id} rejected")
        return 


    get_data = soup.find_all('tr')


    # texts that we have to convert, since it's an image in the table
    txt1 = '<img border="0" src="img/hoshi_shiro.gif"/>'     # win, white circle
    txt2 = '<img border="0" src="img/hoshi_yasumi.gif"/>'    # withdrawal, dash
    txt3 = '<img border="0" src="img/hoshi_kuro.gif"/>'      # loss, black circle
    txt4 = '<img border="0" src="img/hoshi_fusenpai.gif"/>'  # ff, black square
    txt5 = '<img border="0" src="img/hoshi_fusensho.gif"/>' #hoshi_fusensho # withdrawal, white square


    # convert the images to W(win) F(forfeit) L(loss)
    edited = str(get_data).replace(txt1, 'W')
    edited = edited.replace(txt2, 'F')
    edited = edited.replace(txt3, 'L')
    edited = edited.replace(txt4, 'A') # absence 
    edited = edited.replace(txt5, 'W') # enemy ff counts as win


    # put the record table into a dataframe
    test = pd.read_html(edited)
    df = test[2]

    # rename columns
    df.columns = ['Date','Rank', 'Series','W_L','Yusho_standing', 'Height/weight']
    #df['Series'] = df['Series'].replace(np.nan,"")
    # if there is no series, there is no relevant data here most likely (record too old, maezumo)
    df.dropna(subset = ["Series"], inplace=True)

    # drop any possible name banner columns
    df = df[df.W_L.apply(lambda x: '-' in x)]

    # convert dates to string, then datetime

    #df[:, 'Date'] = df["Date"].astype(str)
    df['Date'] = pd.to_datetime(df['Date'].astype(str), format='%Y.%m')

    # we don't really care about yusho standing and height/weight. drop them
    # df['Yusho_standing'] = df['Yusho_standing'].replace(np.nan,"")
    df = df.drop(columns=['Yusho_standing', 'Height/weight'])



    # calculate streaks
    # divide games into 5 sections, if any of the sectios contains W, then
    #series = df['Series'].values

    forfeit_list = []
    win_list = []
    lose_list = []
    ab_list = []
    streak_list = []

    for index, row in df.iterrows():
        f_count = 0
        w_count = 0
        l_count = 0
        a_count = 0
        row_series = row['Series']


        # get W/L/F/A by counting
        for s in row_series:
            if(s == 'F'):
                f_count += 1 
            if(s == 'W'):
                w_count += 1
            if(s == 'L'):
                l_count += 1
            if(s == 'A'):
                a_count += 1

        forfeit_list.append(f_count)
        win_list.append(w_count)
        lose_list.append(l_count)
        ab_list.append(a_count)

        # game is a streak when you win 4 or more games in a row
        streak_list.append(sum(1 for x in re.finditer(r'W{4,}', row['Series'])))






    # make new columns
    df['Win'] = win_list
    df['Loss'] = lose_list
    df['Forfeit'] = forfeit_list
    df['Absence'] = ab_list



    # calculate win rate: (W/# games played), ignoring the games forfeited
    df['Winrate'] = (df['Win']/(15 - (df['Forfeit'] + df['Absence'])))*100
    # remove nans
    df['Winrate'] = df['Winrate'].replace(np.nan,0)
    # round the numbers to the 2 decimal place
    df['Winrate'] = df['Winrate'].round(2)


    # change streak column to Y or N
    # might be a good move to limit the winrate here as well?
    change = []
    for s in streak_list:
        if(s > 0):
            change.append('Y')
        else:
            change.append('N')


    df['Streak(Wins)'] = change
    df.insert(0, 'id', id)
        

    

    # only get the top division tournaments
    return df
        
        

In [7]:
df = get_df_by_id(2933)#.append(get_df_by_id(1), ignore_index=True) 
df.shape

(0, 11)

In [37]:
x = df.loc[df['Streak(Wins)'] == 'Y']
x.head()

mean_wr = x['Winrate'].mean()
mean_wr

# average win rate is 65.91%



test_mean = df['Streak(Wins)'].value_counts().Y
test_mean

# 53 tournaments are streaky



streaky_games = test_mean / len(df)
streaky_games

# 50.48% of the tournaments are streaky, which is good
# but we might have to consider analyzing amount of forfeit/absence numbers, 
# as it could be an interaction term for thrusters vs streakiness

0.6103896103896104

<br>

# DO NOT RUN THIS UNLESS YOU ARE SCRAPING!

In [16]:
# up to 12670
# 2933 empty test case

# for problematic pages
e = []

# go through 0, 13 for everyone
for batch in range(12, 13):
    
    aggregate_df = pd.DataFrame()
    
    for i in range(1 + 1000 * batch, 1000 * (batch+1)):
        print(f"Processing {i}...")
        next = None
        try:
            next = get_df_by_id(i)
        except:
            e.append(i)
        aggregate_df = aggregate_df.append(next, ignore_index=True)

        #df = df[df.Rank.apply(lambda x: rank_helper.rank_is_top(x))]
    
    aggregate_df.to_csv(f'tournaments_all_{batch}.csv', index = False)



Processing 12001...
12001 rejected
Processing 12002...
12002 rejected
Processing 12003...
12003 rejected
Processing 12004...
12004 rejected
Processing 12005...
12005 rejected
Processing 12006...
12006 rejected
Processing 12007...
12007 rejected
Processing 12008...
12008 rejected
Processing 12009...
12009 rejected
Processing 12010...
12010 rejected
Processing 12011...
12011 rejected
Processing 12012...
12012 rejected
Processing 12013...
12013 rejected
Processing 12014...
12014 rejected
Processing 12015...


<ipython-input-4-0f60cf1706ca>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'].astype(str), format='%Y.%m')


Processing 12016...
12016 rejected
Processing 12017...
12017 rejected
Processing 12018...
12018 rejected
Processing 12019...
12019 rejected
Processing 12020...
12020 rejected
Processing 12021...
12021 rejected
Processing 12022...
12022 rejected
Processing 12023...
12023 rejected
Processing 12024...


<ipython-input-4-0f60cf1706ca>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'].astype(str), format='%Y.%m')


Processing 12025...
12025 rejected
Processing 12026...


...

Processing 12454...
12454 rejected
Processing 12455...
12455 rejected
Processing 12456...
12456 rejected
Processing 12457...
12457 rejected
Processing 12458...
12458 rejected
Processing 12459...
12459 rejected
Processing 12460...
12460 rejected
Processing 12461...
12461 rejected
Processing 12462...
12462 rejected
Processing 12463...
12463 rejected
Processing 12464...
12464 rejected
Processing 12465...
12465 rejected
Processing 12466...
12466 rejected
Processing 12467...
12467 rejected
Processing 12468...
12468 rejected
Processing 12469...
12469 rejected
Processing 12470...
Processing 12471...
12471 rejected
Processing 12472...
12472 rejected
Processing 12473...
12473 rejected
Processing 12474...
12474 rejected
Processing 12475...
12475 rejected
Processing 12476...
12476 rejected
Processing 12477...
12477 rejected
Processing 12478...
12478 rejected
Processing 12479...
12479 rejected
Processing 12480...
12480 rejected
Processing 12481...
12481 rejected
Processing 12482...
12482 rejected


<ipython-input-4-0f60cf1706ca>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'].astype(str), format='%Y.%m')


Processing 12576...
12576 rejected
Processing 12577...
12577 rejected
Processing 12578...
12578 rejected
Processing 12579...
12579 rejected
Processing 12580...
12580 rejected
Processing 12581...
12581 rejected
Processing 12582...
12582 rejected
Processing 12583...
12583 rejected
Processing 12584...
12584 rejected
Processing 12585...
12585 rejected
Processing 12586...
12586 rejected
Processing 12587...
12587 rejected
Processing 12588...
12588 rejected
Processing 12589...
12589 rejected
Processing 12590...
12590 rejected
Processing 12591...
12591 rejected
Processing 12592...
12592 rejected
Processing 12593...
12593 rejected
Processing 12594...
12594 rejected
Processing 12595...
12595 rejected
Processing 12596...
12596 rejected
Processing 12597...
12597 rejected
Processing 12598...
12598 rejected
Processing 12599...
12599 rejected
Processing 12600...
12600 rejected
Processing 12601...
12601 rejected
Processing 12602...
12602 rejected
Processing 12603...
12603 rejected
Processing 12604...


12795 skipped prelim
Processing 12796...
12796 skipped prelim
Processing 12797...
12797 skipped prelim
Processing 12798...
12798 skipped prelim
Processing 12799...
12799 skipped prelim
Processing 12800...
12800 skipped prelim
Processing 12801...
12801 skipped prelim
Processing 12802...
12802 skipped prelim
Processing 12803...
12803 skipped prelim
Processing 12804...
12804 skipped prelim
Processing 12805...
12805 skipped prelim
Processing 12806...
12806 skipped prelim
Processing 12807...
12807 skipped prelim
Processing 12808...
12808 skipped prelim
Processing 12809...
12809 skipped prelim
Processing 12810...
12810 skipped prelim
Processing 12811...
12811 skipped prelim
Processing 12812...
12812 skipped prelim
Processing 12813...
12813 skipped prelim
Processing 12814...
12814 skipped prelim
Processing 12815...
12815 skipped prelim
Processing 12816...
12816 skipped prelim
Processing 12817...
12817 skipped prelim
Processing 12818...
12818 skipped prelim
Processing 12819...
12819 skipped pr

12995 skipped prelim
Processing 12996...
12996 skipped prelim
Processing 12997...
12997 skipped prelim
Processing 12998...
12998 skipped prelim
Processing 12999...
12999 skipped prelim


In [17]:
aggregate_df 

,id,Date,Rank,Series,W_L,Win,Loss,Forfeit,Absence,Winrate,Streak(Wins)
0,12015,2012-05-01,Jk8e,FWLFFLLFWFWFWFF,4-3,4,3,8,0,57.14,N
1,12015,2012-07-01,Jd76e,WFFWFWWFFLWFFWF,6-1,6,1,8,0,85.71,N
2,12015,2012-09-01,Jd5w,FLFWFWFWLFWFFLF,4-3,4,3,8,0,57.14,N
3,12015,2012-11-01,Sd86w,FFFFFFFFFFFFFFF,0-0-7,0,0,15,0,0.00,N
4,12015,2013-01-01,Jd47e,LFWFFWWFLFFWFWF,5-2,5,2,8,0,71.43,N
...,...,...,...,...,...,...,...,...,...,...,...
1586,12575,2020-07-01,Ms10w,WFLFFWFWWFWFFFW,6-1,6,1,8,0,85.71,N
1587,12575,2020-09-01,Ms2w,FFFFFFFFFFFFFFF,0-0-7,0,0,15,0,0.00,N
1588,12575,2020-11-01,Ms2w,FLWFWFWFLFLFWFF,4-3,4,3,8,0,57.14,N
1589,12575,2021-01-01,J14w,LWWLLWWLLWWLWWL,8-7,8,7,0,0,53.33,N


In [18]:
# if scraping occured, merge all files
aggregate_df = pd.DataFrame()
for i in range(0, 13):
    if i != 10: # there is nothing in this file.
        df = pd.read_csv(f'tournaments_all_{i}.csv')
        aggregate_df = aggregate_df.append(df)
        
#aggregate_df = aggregate_df.groupby('id').filter(lambda x : len(x) > 5)
aggregate_df.to_csv(f'tournaments_all.csv', index = False)

aggregate_df.shape

(67350, 11)

In [23]:
df1 = pd.read_csv(f'tournaments_all.csv')
df2 = pd.read_csv(f'tournaments_all_copy.csv')

df = df1.append(df2)
df = df.drop_duplicates()
df.to_csv(f'tournaments_all_final.csv', index = False)


***

In [28]:
# load the saved tournament df
df = pd.read_csv(f'tournaments_all_final.csv')
print(df.shape)


# example of selecting only the top two divisions
df = df[df.Rank.apply(lambda x: rank_helper.rank_is_top(x))]
print(df.shape)

(67351, 11)
(34995, 11)


In [29]:
# get the push score (m), push win% (p), and number of wins (n) of a given wrestler by their ID.
def get_m_by_id(id):
    
    kimarite_URL = f"http://sumodb.sumogames.de/Rikishi_kim.aspx?r={id}"
    print(f"processing {id}...")
    basepage = requests.get(kimarite_URL)
    soup = BeautifulSoup(basepage.content, 'html.parser')

    # get winning data table 
    kimarite_wins = soup.find('table', class_="ro_left")

    # this gets all winning moves in like [<tr><td class="rb_kim">oshidashi</td><td class="right"><a href="#0oshidashi">143</a></td></tr>, ...]
    cells = kimarite_wins.find_all('tr')

    kimarite_labels, kimarite_amts = [], []

    for cell in cells:
        win_name_tag, win_amt_tag = cell.find_all('td')
        kimarite_labels.append(win_name_tag.text.strip())
        kimarite_amts.append(int(win_amt_tag.text.strip()))


    # at this point we have all kimarite. reduce it so that only moves with >= 5% significance are displayed
    up_to_index = 0
    total_wins = sum(kimarite_amts)
    for i in range(0, len(kimarite_amts)):
        amt = kimarite_amts[i]
        # we find the most significant kimarite under 10%
        if amt / total_wins < 0.05:
            up_to_index = i
            break

    # we now slice off everything up to and past this point, label it as 'other'
    kimarite_labels = kimarite_labels[:up_to_index] + ["other"]
    kimarite_amts = kimarite_amts[:up_to_index] + [sum(kimarite_amts[up_to_index:])]

    #print(kimarite_labels)
    #print(kimarite_amts)

    #print((shikona, kimarite_amts, kimarite_labels))
    
    
    n = sum(kimarite_amts)
    if n == 0:
        return (None, None, None)
        
    w = 0 
    for i in range(0, len(kimarite_amts)):
        w += kimarite_amts[i] if kimarite_helper.is_oshi(kimarite_labels[i]) else 0
    
    p = w / n
    m = (0.5 - p)  * math.log10(n)
    return (m, p, n)



In [30]:
# filter out the wrestlers with less than five tournaments
#df = df.groupby('id').filter(lambda x : len(x) > 5)

stats_df = pd.DataFrame(columns = ["id", "m", "p", "n"])
d = []

for i in df.id.unique():
    
    m, p, n = get_m_by_id(i)
    if m:
        stats_df = stats_df.append({'id': i, 'm': m, 'p': p, 'n': n}, ignore_index=True)
    else:
        d.append(i)


stats_df

processing 1...
processing 2...
processing 3...
processing 4...
processing 5...
processing 6...
processing 7...
processing 8...
processing 9...
processing 10...
processing 11...
processing 12...
processing 13...
processing 14...
processing 15...
processing 16...
processing 17...
processing 18...
processing 19...
processing 20...
processing 21...
processing 22...
processing 23...
processing 24...
processing 25...
processing 26...
processing 27...
processing 28...
processing 29...
processing 30...
processing 31...
processing 32...
processing 33...
processing 34...
processing 35...
processing 36...
processing 37...
processing 38...
processing 39...
processing 40...
processing 41...
processing 42...
processing 43...
processing 44...
processing 45...
processing 46...
processing 47...
processing 48...
processing 49...
processing 50...
processing 51...
processing 52...
processing 53...
processing 54...
processing 55...
processing 56...
processing 57...
processing 58...
processing 59...
proces

processing 3690...
processing 3691...
processing 3692...
processing 3693...
processing 3694...
processing 3695...
processing 3696...
processing 3697...
processing 3698...
processing 3699...
processing 3700...
processing 3701...
processing 3702...
processing 3703...
processing 3704...
processing 3705...
processing 3706...
processing 3707...
processing 3708...
processing 3709...
processing 3710...
processing 3711...
processing 3712...
processing 3713...
processing 3714...
processing 3715...
processing 3716...
processing 3717...
processing 3718...
processing 3719...
processing 3720...
processing 3721...
processing 3722...
processing 3723...
processing 3724...
processing 3725...
processing 3726...
processing 3727...
processing 3728...
processing 3729...
processing 3730...
processing 3731...
processing 3732...
processing 3733...
processing 3734...
processing 3735...
processing 3736...
processing 3737...
processing 3738...
processing 3739...
processing 3740...
processing 3741...
processing 3

processing 4123...
processing 4124...
processing 4125...
processing 4126...
processing 4127...
processing 4128...
processing 4129...
processing 4130...
processing 4131...
processing 4132...
processing 4133...
processing 4134...
processing 4135...
processing 4136...
processing 4137...
processing 4138...
processing 4139...
processing 4140...
processing 4141...
processing 4142...
processing 4143...
processing 4144...
processing 4145...
processing 4146...
processing 4147...
processing 4148...
processing 4149...
processing 4150...
processing 4151...
processing 4152...
processing 4153...
processing 4154...
processing 4155...
processing 4156...
processing 4157...
processing 4158...
processing 4159...
processing 4160...
processing 4161...
processing 4162...
processing 4163...
processing 4164...
processing 4165...
processing 4166...
processing 4167...
processing 4168...
processing 4169...
processing 4170...
processing 4171...
processing 4172...
processing 4173...
processing 4174...
processing 4

processing 6410...
processing 6411...
processing 6419...
processing 6422...
processing 6427...
processing 6429...
processing 6432...
processing 6463...
processing 6467...
processing 6468...
processing 6473...
processing 6480...
processing 6491...
processing 6497...
processing 6512...
processing 6527...
processing 6531...
processing 6552...
processing 6559...
processing 6572...
processing 6594...
processing 6596...
processing 6599...
processing 6600...
processing 6614...
processing 6620...
processing 6622...
processing 6642...
processing 6654...
processing 6753...
processing 6766...
processing 6767...
processing 6769...
processing 6770...
processing 6771...
processing 6928...
processing 6931...
processing 6977...
processing 6983...
processing 7109...
processing 7111...
processing 7115...
processing 7121...
processing 7125...
processing 7127...
processing 7138...
processing 7143...
processing 7153...
processing 7239...
processing 7240...
processing 8154...
processing 8899...
processing 8

,id,m,p,n
0,1.0,0.027626,0.490066,604.0
1,2.0,1.220098,0.075798,752.0
2,3.0,0.536245,0.302682,522.0
3,4.0,0.160073,0.444142,734.0
4,5.0,1.253818,0.058140,688.0
...,...,...,...,...
1075,12449.0,-0.099929,0.548673,113.0
1076,12451.0,0.707805,0.156522,115.0
1077,12453.0,-0.389437,0.702381,84.0
1078,12470.0,-0.700176,0.865854,82.0


In [32]:
stats_df.to_csv(f'stats_by_id.csv', index = False)

things to note
* only analyzing those who made it to the top division
* not analyzing old records (no proof for push/non-push)
* classifying pushers only by winning techniques

things to explore
* bouts in certain time periods
* bouts in top divisions only (sanyaku, makuuchi, etc.)

big questions
* should we deal with wrestlers like akebono differently? is the m score representative?
* body weight/size of wrestler as opposed to their style